My research paper that needs to be implemented was to Provably Robust Deep Learning via Adversarially
Trained Smoothed Classifiers<br> 

Paper Link- https://arxiv.org/pdf/1906.04584.pdf



# Importing Libraries

In [1]:
pip install git+https://github.com/CNOCycle/cleverhans.git@feature/tf2.x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/CNOCycle/cleverhans.git (to revision feature/tf2.x) to /tmp/pip-req-build-rrb0smpw
  Running command git clone --filter=blob:none --quiet https://github.com/CNOCycle/cleverhans.git /tmp/pip-req-build-rrb0smpw
  Running command git checkout -b feature/tf2.x --track origin/feature/tf2.x
  Switched to a new branch 'feature/tf2.x'
  Branch 'feature/tf2.x' set up to track remote branch 'feature/tf2.x' from 'origin'.
  Resolved https://github.com/CNOCycle/cleverhans.git to commit b103b0caf983331ec9c5f7c3e08e417a85bf25e2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 KB 5.8 MB/s eta 0:00:00
  Created wheel for cleverhans: filename=cleverhans-3.0.1-py3-none-any.whl size=254398 sha256=d997afb171eca80488ca365112ead9181d0fbaa7f743f2b240caafed

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from PIL import Image

import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset 
from torchvision.datasets import CIFAR10,MNIST
from torchvision import transforms 
from torch.utils.data import random_split
from torchvision.utils import make_grid
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.utils.data import Subset
from cleverhans.future.torch.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.future.torch.attacks.projected_gradient_descent import projected_gradient_descent


from absl import app, flags
import time 
import copy
import tqdm
import os
import io
from collections import Counter
%matplotlib inline

In [3]:
FLAGS = flags.FLAGS

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Getting the data

In [5]:
#making the transform for our dataset 
transform=transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()])

In [6]:
train_dataset = CIFAR10(root = 'data/', download = True, transform = transform)
test_dataset = CIFAR10(root = 'data/', train = False, transform = transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data/


In [7]:
print(train_dataset) 
print(test_dataset)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data/
    Split: Train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: data/
    Split: Test
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )


In [8]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                         batch_size=8, 
                                         drop_last=True)

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                         batch_size=8, 
                                         drop_last=True)

In [9]:
#making the datalaoder dictionary 
dataloaders_dict = {"train": train_dataloader, "val": test_dataloader}

# Training a normal classifier

## Defining a normal CNN

In [ ]:
class CNN_normal(nn.Module): 
  def __init__(self): 
    super().__init__()
    self.cnn=nn.Sequential(nn.Conv2d(3,32,kernel_size=3,padding=1),
                           nn.ReLU(True),
                           nn.MaxPool2d(2,2),
                           nn.Conv2d(32,64,kernel_size=3,padding=1),
                           nn.ReLU(True),
                           nn.MaxPool2d(2,2),
                           nn.Conv2d(64,64,kernel_size=3,padding=1),
                           nn.ReLU(),
                           nn.MaxPool2d(2,2)
                           )
    
    self.linear_layer=nn.Sequential(nn.Flatten(),
                                    nn.Linear(64*4*4,100),
                                    nn.Linear(100,10))
    
  def forward(self,x): 
    x=self.cnn(x)
    x=self.linear_layer(x)

    return x

In [ ]:
model_2=CNN_normal()
print(model_2)

CNN_normal(
  (cnn): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layer): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1024, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)


## Training the classifier

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, device='cpu',noise_factor=0.2):
      since = time.time()

      best_model_wts = copy.deepcopy(model.state_dict())
      best_acc = 0.0
      model.to(device)

      for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
          if phase == 'train':
            model.train()  # Set model to training mode
          else:
            model.eval()   # Set model to evaluate mode

          running_loss = 0.0
          running_corrects = 0

          # Iterate over data.
          for inputs, labels in dataloaders_dict[phase]:
            #adding noise to my image
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
              outputs = model(inputs)
              _, preds = torch.max(outputs, 1)
              loss = criterion(outputs, labels)

              # backward + optimize only if in training phase
              if phase == 'train':
                loss.backward()
                optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
          if phase == 'train':
            scheduler.step()

          epoch_loss = running_loss / len(dataloaders_dict[phase].dataset)
          epoch_acc = running_corrects.double() / len(dataloaders_dict[phase].dataset)

          print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))

          # deep copy the model
          if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        print()

      time_elapsed = time.time() - since
      print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
      print('Best val Acc: {:4f}'.format(best_acc))

      # load best model weights
      model.load_state_dict(best_model_wts)
      return model

In [ ]:
criterion=torch.nn.CrossEntropyLoss()
learning_rate=0.01 
optimizer_2 = torch.optim.SGD(model_2.parameters(), lr=learning_rate) 
exp_lr_scheduler_2 = optim.lr_scheduler.StepLR(optimizer_2, step_size=7, gamma=0.1)

In [ ]:
original_model = train_model(model_2, criterion, optimizer_2, exp_lr_scheduler_2,
                        num_epochs=10, device='cuda')

Epoch 0/9
----------
train Loss: 1.8677 Acc: 0.3169
val Loss: 1.5499 Acc: 0.4329

Epoch 1/9
----------
train Loss: 1.4019 Acc: 0.4991
val Loss: 1.2742 Acc: 0.5445

Epoch 2/9
----------
train Loss: 1.1695 Acc: 0.5884
val Loss: 1.1012 Acc: 0.6123

Epoch 3/9
----------
train Loss: 1.0279 Acc: 0.6421
val Loss: 1.0260 Acc: 0.6403

Epoch 4/9
----------
train Loss: 0.9341 Acc: 0.6757
val Loss: 0.9093 Acc: 0.6862

Epoch 5/9
----------
train Loss: 0.8631 Acc: 0.6996
val Loss: 0.8722 Acc: 0.6968

Epoch 6/9
----------
train Loss: 0.8129 Acc: 0.7170
val Loss: 0.8437 Acc: 0.7102

Epoch 7/9
----------
train Loss: 0.6810 Acc: 0.7659
val Loss: 0.7697 Acc: 0.7353

Epoch 8/9
----------
train Loss: 0.6593 Acc: 0.7735
val Loss: 0.7596 Acc: 0.7411

Epoch 9/9
----------
train Loss: 0.6460 Acc: 0.7773
val Loss: 0.7491 Acc: 0.7441

Training complete in 4m 37s
Best val Acc: 0.744100


In [ ]:
#saving my model 
torch.save(original_model,'/content/drive/MyDrive/pre trained models/original_CNN_Cifar10.pb')

## Testing it on adversarial attack

This helps to attack the image with some adervsial attack with the given epsilon

In [18]:
# FGSM attack code
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon*sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

This function applies the adverisal attack on the validation data and return the average accuracy 

In [19]:
def evalAdvAttack(epsilon,fgsm_model=None, test_loader=None):
  total=0 
  correct=0 
  #looping over all the test data 
  for data,target in test_loader:
    data, target = data.to(device), target.to(device)
    #important for the attack
    data.requires_grad = True
    #collecting the data gradient
    #adding attack to the image using the fgsm_attack function 
    # perturbed_data = fgsm_attack(data, epsilon, data) 
    perturbed_data = fast_gradient_method(fgsm_model, data, eps=epsilon, norm=np.inf, clip_min=0., clip_max=1.)
    #sending the data to the model 
    preds1 = fgsm_model(perturbed_data)
    preds_np1 = preds1.cpu().detach().numpy()
    finalPred = np.argmax(preds_np1, axis=1)
    correct += (finalPred == target.cpu().detach().numpy()).sum()
    total += test_loader.batch_size
  acc = float(correct) / total
  print('Adv accuracy: {:.3f}％'.format(acc * 100))

In [ ]:
evalAdvAttack(0.05,fgsm_model=original_model, test_loader=test_dataloader)

Adv accuracy: 12.250％


In [ ]:
evalAdvAttack(0.2,fgsm_model=original_model, test_loader=test_dataloader)

Adv accuracy: 8.570％


In [ ]:
evalAdvAttack(0.3,fgsm_model=original_model, test_loader=test_dataloader)

Adv accuracy: 8.590％


Our original accuracy was 74% but with value of episolon value of 0.05- 12.250% and with 0.2 it has the value of8.57% and with 0.3 it is 0.8590 <br> 
Which means our original model is not prune to the adverisal attacks 

# Training a smoothened classifier
A smoothened classifier means training a classifier with some noise in data so that it is prune against small perbutations in data<br> 
However it is not that much effective against adervsial attacks<br> 
Lets see it

## Defining a simple CNN

In [ ]:
class CNN(nn.Module): 
  def __init__(self): 
    super().__init__()
    self.cnn=nn.Sequential(nn.Conv2d(3,32,kernel_size=3,padding=1),
                           nn.ReLU(True),
                           nn.MaxPool2d(2,2),
                           nn.Conv2d(32,64,kernel_size=3,padding=1),
                           nn.ReLU(True),
                           nn.MaxPool2d(2,2),
                           nn.Conv2d(64,64,kernel_size=3,padding=1),
                           nn.ReLU(),
                           nn.MaxPool2d(2,2)
                           )
    
    self.linear_layer=nn.Sequential(nn.Flatten(),
                                    nn.Linear(64*4*4,100),
                                    nn.Linear(100,10))
    
  def forward(self,x): 
    x=self.cnn(x)
    x=self.linear_layer(x)

    return x

In [ ]:
model_1=CNN()
print(model_1)

CNN(
  (cnn): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layer): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1024, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)


##  Smooth Training the model

In [ ]:
def train_smooth_model(model, criterion, optimizer, scheduler, num_epochs=25, device='cpu',noise_factor=0.2):
      since = time.time()

      best_model_wts = copy.deepcopy(model.state_dict())
      best_acc = 0.0
      model.to(device)

      for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
          if phase == 'train':
            model.train()  # Set model to training mode
          else:
            model.eval()   # Set model to evaluate mode

          running_loss = 0.0
          running_corrects = 0

          # Iterate over data.
          for inputs, labels in dataloaders_dict[phase]:
            #adding noise to my image
            img_noisy = inputs + noise_factor * torch.randn(inputs.shape)
            inputs = inputs.to(device)
            labels = labels.to(device)
            img_noisy=img_noisy.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
              outputs = model(img_noisy)
              _, preds = torch.max(outputs, 1)
              loss = criterion(outputs, labels)

              # backward + optimize only if in training phase
              if phase == 'train':
                loss.backward()
                optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
          if phase == 'train':
            scheduler.step()

          epoch_loss = running_loss / len(dataloaders_dict[phase].dataset)
          epoch_acc = running_corrects.double() / len(dataloaders_dict[phase].dataset)

          print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))

          # deep copy the model
          if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        print()

      time_elapsed = time.time() - since
      print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
      print('Best val Acc: {:4f}'.format(best_acc))

      # load best model weights
      model.load_state_dict(best_model_wts)
      return model

In [ ]:
criterion=torch.nn.CrossEntropyLoss()
learning_rate=0.01 
optimizer_1 = torch.optim.SGD(model_1.parameters(), lr=learning_rate) 
exp_lr_scheduler_1 = optim.lr_scheduler.StepLR(optimizer_1, step_size=7, gamma=0.1)

In [ ]:
new_model = train_smooth_model(model_1, criterion, optimizer_1, exp_lr_scheduler_1,
                        num_epochs=10, device='cuda')

Epoch 0/9
----------
train Loss: 1.7697 Acc: 0.3606
val Loss: 1.5271 Acc: 0.4447

Epoch 1/9
----------
train Loss: 1.4657 Acc: 0.4747
val Loss: 1.3723 Acc: 0.5041

Epoch 2/9
----------
train Loss: 1.3249 Acc: 0.5274
val Loss: 1.2666 Acc: 0.5521

Epoch 3/9
----------
train Loss: 1.2250 Acc: 0.5684
val Loss: 1.1889 Acc: 0.5749

Epoch 4/9
----------
train Loss: 1.1567 Acc: 0.5933
val Loss: 1.1366 Acc: 0.5976

Epoch 5/9
----------
train Loss: 1.1094 Acc: 0.6104
val Loss: 1.1110 Acc: 0.6045

Epoch 6/9
----------
train Loss: 1.0694 Acc: 0.6251
val Loss: 1.0696 Acc: 0.6276

Epoch 7/9
----------
train Loss: 0.9668 Acc: 0.6622
val Loss: 1.0143 Acc: 0.6452

Epoch 8/9
----------
train Loss: 0.9483 Acc: 0.6680
val Loss: 1.0059 Acc: 0.6503

Epoch 9/9
----------
train Loss: 0.9405 Acc: 0.6712
val Loss: 0.9895 Acc: 0.6524

Training complete in 5m 5s
Best val Acc: 0.652400


Saving my model for further use 

In [ ]:
torch.save(new_model,'/content/drive/MyDrive/pre trained models/smoothened_CNN_Cifar10.pb')

## Testing it on adevrsial attack 



now applying the adversial attack on my test data using test dataloader to see the accuracy drop 

Running the attack with a epsilon value of 0.2

In [ ]:
evalAdvAttack(0.05,fgsm_model=new_model, test_loader=test_dataloader)

Adv accuracy: 20.090％


In [ ]:
evalAdvAttack(0.2,fgsm_model=new_model, test_loader=test_dataloader)

Adv accuracy: 17.310％


Now checking it epsilon value of 0.3

In [ ]:
evalAdvAttack(0.3,fgsm_model=new_model, test_loader=test_dataloader)

Adv accuracy: 14.970％


We can see that with the smoothening training the accuracy the adv accuracy at 0.05 is 20% , at 0.2 it is 17%, but with 0.3% it is 14% which means it is better than the original model but still far than achieving any meaniningful results

# Adversially training a smoothned classifer

## Defining a simple CNN

In [14]:
class CNN(nn.Module): 
  def __init__(self): 
    super().__init__()
    self.cnn=nn.Sequential(nn.Conv2d(3,32,kernel_size=3,padding=1),
                           nn.ReLU(True),
                           nn.MaxPool2d(2,2),
                           nn.Conv2d(32,64,kernel_size=3,padding=1),
                           nn.ReLU(True),
                           nn.MaxPool2d(2,2),
                           nn.Conv2d(64,64,kernel_size=3,padding=1),
                           nn.ReLU(),
                           nn.MaxPool2d(2,2)
                           )
    
    self.linear_layer=nn.Sequential(nn.Flatten(),
                                    nn.Linear(64*4*4,100),
                                    nn.Linear(100,10))
    
  def forward(self,x): 
    x=self.cnn(x)
    x=self.linear_layer(x)

    return x

In [ ]:
model_3=CNN()
print(model_3)

CNN(
  (cnn): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layer): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1024, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)


## Adversially training a smoothned classifier
In adverisally training a smooth classifier we also take some of the adverisal data with our training <br> 


In [ ]:
def train_adversial_smooth_model(model, criterion1,criterion2, optimizer, scheduler, num_epochs=25, device='cpu',noise_factor=0.2,epsilon=0.2):
      since = time.time()

      best_model_wts = copy.deepcopy(model.state_dict())
      best_acc = 0.0
      model.to(device)

      for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
          if phase == 'train':
            model.train()  # Set model to training mode
          else:
            model.eval()   # Set model to evaluate mode

          running_loss = 0.0
          running_corrects = 0

          # Iterate over data.
          for inputs, labels in dataloaders_dict[phase]:
            #adding noise to my image
            img_noisy = inputs + noise_factor * torch.randn(inputs.shape)
            inputs = inputs.to(device)
            labels = labels.to(device)
            img_noisy=img_noisy.to(device)

            perturbed_data = fast_gradient_method(model, img_noisy, eps=epsilon, norm=np.inf, clip_min=0., clip_max=1.)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
              outputs1 = model(img_noisy)
              loss1 = criterion1(outputs1, labels)

              #now appyling the adverisal attack on my data 
              outputs2 = model(perturbed_data)
              _, preds2 = torch.max(outputs2, 1)
              loss2 = criterion2(outputs2, labels)

              #making a final loss by comining both the loss 
              final_loss=0.5*loss1+0.5*loss2


              # backward + optimize only if in training phase
              if phase == 'train':
                final_loss.backward()
                optimizer.step()

            # statistics
            running_loss += final_loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds2 == labels.data)
          if phase == 'train':
            scheduler.step()

          epoch_loss = running_loss / len(dataloaders_dict[phase].dataset)
          epoch_acc = running_corrects.double() / len(dataloaders_dict[phase].dataset)

          print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))

          # deep copy the model
          if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        print()

      time_elapsed = time.time() - since
      print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
      print('Best val Acc: {:4f}'.format(best_acc))

      # load best model weights
      model.load_state_dict(best_model_wts)
      return model

In [ ]:
criterion1=torch.nn.CrossEntropyLoss()
criterion2=torch.nn.CrossEntropyLoss()
learning_rate=0.01 
optimizer_3 = torch.optim.SGD(model_3.parameters(), lr=learning_rate) 
exp_lr_scheduler_3 = optim.lr_scheduler.StepLR(optimizer_3, step_size=7, gamma=0.1)

In [ ]:
adeversial_smooth_model = train_adversial_smooth_model(model_1, criterion1,criterion2, optimizer_1, exp_lr_scheduler_1,
                        num_epochs=20, device='cuda',epsilon=0.2)

Epoch 0/19
----------
train Loss: 1.9996 Acc: 0.1788
val Loss: 1.8906 Acc: 0.1911

Epoch 1/19
----------
train Loss: 1.8484 Acc: 0.1851
val Loss: 1.8035 Acc: 0.1973

Epoch 2/19
----------
train Loss: 1.7787 Acc: 0.1941
val Loss: 1.7539 Acc: 0.2058

Epoch 3/19
----------
train Loss: 1.7413 Acc: 0.1978
val Loss: 1.7336 Acc: 0.2107

Epoch 4/19
----------
train Loss: 1.7176 Acc: 0.2061
val Loss: 1.7113 Acc: 0.2081

Epoch 5/19
----------
train Loss: 1.6998 Acc: 0.2065
val Loss: 1.6988 Acc: 0.2127

Epoch 6/19
----------
train Loss: 1.6867 Acc: 0.2131
val Loss: 1.6864 Acc: 0.2176

Epoch 7/19
----------
train Loss: 1.6805 Acc: 0.2157
val Loss: 1.6877 Acc: 0.2188

Epoch 8/19
----------
train Loss: 1.6800 Acc: 0.2134
val Loss: 1.6836 Acc: 0.2187

Epoch 9/19
----------
train Loss: 1.6782 Acc: 0.2155
val Loss: 1.6814 Acc: 0.2218

Epoch 10/19
----------
train Loss: 1.6780 Acc: 0.2171
val Loss: 1.6814 Acc: 0.2188

Epoch 11/19
----------
train Loss: 1.6777 Acc: 0.2146
val Loss: 1.6797 Acc: 0.2220

Ep

As we can see that our best accuracy is 0.22 for epislon training at 0.2 so reducing the epsilon value for some more meaningful results

In [ ]:
adeversial_smooth_model_1 = train_adversial_smooth_model(model_1, criterion1,criterion2, optimizer_1, exp_lr_scheduler_1,
                        num_epochs=20, device='cuda',epsilon=0.01)

Epoch 0/19
----------
train Loss: 1.2870 Acc: 0.4857
val Loss: 1.3019 Acc: 0.4827

Epoch 1/19
----------
train Loss: 1.2873 Acc: 0.4883
val Loss: 1.3094 Acc: 0.4741

Epoch 2/19
----------
train Loss: 1.2840 Acc: 0.4884
val Loss: 1.3019 Acc: 0.4849

Epoch 3/19
----------
train Loss: 1.2837 Acc: 0.4881
val Loss: 1.3030 Acc: 0.4824

Epoch 4/19
----------
train Loss: 1.2823 Acc: 0.4879
val Loss: 1.2969 Acc: 0.4846

Epoch 5/19
----------
train Loss: 1.2806 Acc: 0.4863
val Loss: 1.2978 Acc: 0.4808

Epoch 6/19
----------
train Loss: 1.2809 Acc: 0.4880
val Loss: 1.2923 Acc: 0.4823

Epoch 7/19
----------
train Loss: 1.2791 Acc: 0.4898
val Loss: 1.2948 Acc: 0.4785

Epoch 8/19
----------
train Loss: 1.2781 Acc: 0.4892
val Loss: 1.2925 Acc: 0.4933

Epoch 9/19
----------
train Loss: 1.2774 Acc: 0.4893
val Loss: 1.2958 Acc: 0.4774

Epoch 10/19
----------
train Loss: 1.2812 Acc: 0.4895
val Loss: 1.3003 Acc: 0.4760

Epoch 11/19
----------
train Loss: 1.2779 Acc: 0.4931
val Loss: 1.3070 Acc: 0.4813

Ep

## Testing it on adverisal attack

In [ ]:
evalAdvAttack(0.2,fgsm_model=adeversial_smooth_model_1, test_loader=test_dataloader)

Adv accuracy: 26.950％


In [ ]:
evalAdvAttack(0.3,fgsm_model=adeversial_smooth_model_1, test_loader=test_dataloader)

Adv accuracy: 26.000％


As we can see that by training my model with epilonm at 0.01 the accuracy for adversial attack on test data with 0.2 and 0.3 is much better than the previous ones

Now training with the model that was trained with epsilon at 0.2

In [ ]:
evalAdvAttack(0.2,fgsm_model=adeversial_smooth_model, test_loader=test_dataloader)

Adv accuracy: 27.350％


In [ ]:
evalAdvAttack(0.3,fgsm_model=adeversial_smooth_model, test_loader=test_dataloader)

Adv accuracy: 26.160％


As we can see that the accuracy was almost the same as above

# Adevrsially training a smoothened classifier with l2 regulariation

## Making our model

In [12]:
model_4=CNN()
print(model_4)

CNN(
  (cnn): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layer): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1024, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)


## Adverisally training with l2 regularlization

In [6]:
def pgd_attack(model, x, y, epsilon, alpha, num_iters):
    """Generate adversarial examples using PGD."""
    delta = torch.zeros_like(x).uniform_(-epsilon, epsilon)
    for t in range(num_iters):
        delta.requires_grad_()
        with torch.enable_grad():
            loss = nn.CrossEntropyLoss()(model(x + delta), y)
        grad = torch.autograd.grad(loss, delta)[0]
        delta = (delta + alpha * grad.detach().sign()).clamp_(-epsilon, epsilon)
        delta = torch.min(torch.max(delta, -x), 1 - x)  # ensure x + delta is within [0, 1]
    return x + delta

In [7]:
def train_adversial_smooth_model(model, criterion1,criterion2, optimizer, scheduler, num_epochs=25, device='cpu',noise_factor=0.2,epsilon=0.2,alpha=0.05,num_iters=10,sigma=0.05):
      since = time.time()

      best_model_wts = copy.deepcopy(model.state_dict())
      best_acc = 0.0
      model.to(device)

      for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
          if phase == 'train':
            model.train()  # Set model to training mode
          else:
            model.eval()   # Set model to evaluate mode

          running_loss = 0.0
          running_corrects = 0

          # Iterate over data.
          for inputs, labels in dataloaders_dict[phase]:
            #adding noise to my image
            img_noisy = inputs + noise_factor * torch.randn(inputs.shape)
            inputs = inputs.to(device)
            labels = labels.to(device)
            img_noisy=img_noisy.to(device)
            img_noisy.requires_grad=True

            perturbed_data = pgd_attack(model, inputs, labels, epsilon, alpha, num_iters)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
              outputs1 = model(img_noisy)
              loss1 = criterion1(outputs1, labels)

              #now appyling the adverisal attack on my data 
              outputs2 = model(perturbed_data)
              _, preds2 = torch.max(outputs2, 1)
              loss2 = criterion2(outputs2, labels)

              #making a final loss by comining both the loss 


              # backward + optimize only if in training phase
              if phase == 'train':
                grad = torch.autograd.grad(loss1, img_noisy,retain_graph=True)[0]
                loss_smooth = loss1 + sigma**2 / 2 * (grad.norm()**2 + (perturbed_data - inputs).norm()**2)
                final_loss = loss2+ loss_smooth
                final_loss.backward()
                optimizer.step()

            # statistics
            running_loss += final_loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds2 == labels.data)
          if phase == 'train':
            scheduler.step()

          epoch_loss = running_loss / len(dataloaders_dict[phase].dataset)
          epoch_acc = running_corrects.double() / len(dataloaders_dict[phase].dataset)

          print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))

          # deep copy the model
          if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        print()

      time_elapsed = time.time() - since
      print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
      print('Best val Acc: {:4f}'.format(best_acc))

      # load best model weights
      model.load_state_dict(best_model_wts)
      return model

In [41]:
criterion1=torch.nn.CrossEntropyLoss()
criterion2=torch.nn.CrossEntropyLoss()
learning_rate=0.01 
optimizer_4 = torch.optim.SGD(model_4.parameters(), lr=learning_rate) 
exp_lr_scheduler_4 = optim.lr_scheduler.StepLR(optimizer_4, step_size=7, gamma=0.1)

In [42]:
adeversial_smooth_lr_model_1 = train_adversial_smooth_model(model_4, criterion1,criterion2, optimizer_4, exp_lr_scheduler_4,
                        num_epochs=10, device='cuda',epsilon=0.01)

Epoch 0/9
----------
train Loss: 3.1982 Acc: 0.3584
val Loss: 4.8238 Acc: 0.3697

Epoch 1/9
----------
train Loss: 3.0364 Acc: 0.3827
val Loss: 4.5617 Acc: 0.3826

Epoch 2/9
----------
train Loss: 2.9217 Acc: 0.3995
val Loss: 4.6720 Acc: 0.4005

Epoch 3/9
----------
train Loss: 2.8404 Acc: 0.4130
val Loss: 4.4900 Acc: 0.4083

Epoch 4/9
----------
train Loss: 2.7745 Acc: 0.4232
val Loss: 4.5468 Acc: 0.4220

Epoch 5/9
----------
train Loss: 2.7186 Acc: 0.4300
val Loss: 3.8489 Acc: 0.4305

Epoch 6/9
----------
train Loss: 2.6636 Acc: 0.4393
val Loss: 3.7708 Acc: 0.4405

Epoch 7/9
----------
train Loss: 2.4350 Acc: 0.4721
val Loss: 3.2044 Acc: 0.4686

Epoch 8/9
----------
train Loss: 2.4001 Acc: 0.4792
val Loss: 3.3858 Acc: 0.4689

Epoch 9/9
----------
train Loss: 2.3895 Acc: 0.4800
val Loss: 2.9294 Acc: 0.4721

Training complete in 29m 53s
Best val Acc: 0.472100


In [45]:
torch.save(adeversial_smooth_lr_model_1,'/content/drive/MyDrive/adversial_smoothened_lr_CNN_Cifar10.pb')

## Testing it on adversial attack

In [8]:
# Define the test function
def test(model, device, test_loader, epsilon, alpha, num_iter):
    # Set the model to evaluation mode
    model.eval()

    # Define the adversarial attack object

    # Define variables to keep track of the test accuracy and the number of correctly classified samples
    test_loss = 0
    correct = 0

    # Iterate over the test data
    for data, target in test_loader:
        # Move the data and target tensors to the device
        data, target = data.to(device), target.to(device)

        # Generate adversarial examples using PGD attack
        perturbed_data = pgd_attack(model, data, target,epsilon=epsilon, alpha=alpha, num_iters=num_iter)

        # Compute the model's predictions on the adversarial examples
        output = model(perturbed_data)

        # Compute the test loss
        test_loss += F.nll_loss(output, target, reduction='sum').item()

        # Get the index of the predicted class
        pred = output.argmax(dim=1, keepdim=True)

        # Update the number of correctly classified samples
        correct += pred.eq(target.view_as(pred)).sum().item()

    # Compute the final test accuracy
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset), test_accuracy))

    return test_accuracy

As you can see that our model even with epsilon at 0.2 we are performing good 

In [52]:
test(epsilon=0.2,alpha=0.05,num_iter=10,model=adeversial_smooth_lr_model_1, test_loader=test_dataloader,device='cuda')

Test set: Average loss: 0.1353, Accuracy: 4236/10000 (42.36%)


# Adversially Training Renset18 with l2 regularization
For this pupose I will be using MNIST as due to heavy computation problem in cifar 10 

## Getting the MNIST data

In [2]:
#making the transform for our dataset 
transform=transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()])

In [3]:
train_dataset = MNIST(root = 'data/', download = True, transform = transform)
test_dataset = MNIST(root = 'data/', train = False, transform = transform)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [4]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                         batch_size=8, 
                                         drop_last=True)

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                         batch_size=8, 
                                         drop_last=True)

In [5]:
#making the datalaoder dictionary 
dataloaders_dict = {"train": train_dataloader, "val": test_dataloader}

## Getting the Resnet18 model

In [14]:
resnet18 = models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features

In [15]:
print(resnet18)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) 

In [20]:
#now we have the output features equal to 10 so we need to make output layers to be 10 
resnet18.fc = nn.Linear(num_ftrs, 10)

## Adversially Training resnet18 

In [21]:
criterion1=torch.nn.CrossEntropyLoss()
criterion2=torch.nn.CrossEntropyLoss()
learning_rate=0.01 
optimizer_5 = torch.optim.SGD(resnet18.parameters(), lr=learning_rate) 
exp_lr_scheduler_5 = optim.lr_scheduler.StepLR(optimizer_5, step_size=7, gamma=0.1)

In [29]:
adeversial_smooth_lr_model_1 = train_adversial_smooth_model(resnet18, criterion1,criterion2, optimizer_5, exp_lr_scheduler_5,
                        num_epochs=10, device='cuda',epsilon=0.01)

Epoch 0/9
----------
train Loss: 1.0017 Acc: 0.8448
val Loss: 0.0646 Acc: 0.9435

Epoch 1/9
----------
train Loss: 0.2927 Acc: 0.9541
val Loss: 0.0118 Acc: 0.9697

Epoch 2/9
----------
train Loss: 0.1955 Acc: 0.9688
val Loss: 0.0073 Acc: 0.9746

Epoch 3/9
----------
train Loss: 0.1448 Acc: 0.9760
val Loss: 0.0021 Acc: 0.9383

Epoch 4/9
----------



The output was taking for ever so I stopped it beause I already has a good accuracy of 0.97 on test dataset 

## testing it on adverisal attacks

In [34]:

test(epsilon=0.2,alpha=0.05,num_iter=10,model=resnet18, test_loader=test_dataloader,device='cuda')

Test set: Average loss: 0.765, Accuracy: 9236/10000 (92.36%)


In [32]:
test(epsilon=0.3,alpha=0.05,num_iter=10,model=resnet18, test_loader=test_dataloader,device='cuda')

Test set: Average loss: 0.821, Accuracy: 8978/10000 (89.78%)
